<a href="https://colab.research.google.com/github/ykitaguchi77/manipulate_CSV/blob/master/Anonymous_connect2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**匿名データの連結**

【やること】<br>

PatientInfoのリストに、IDMapファイルを用いてIDを連結<br>
さらにDiseaseInfoファイルから病名を連結<br>
DiseaseInfo_all_connectedに含まれていないリストのみを抜粋して通し番号をつける

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#**各CSVファイルを開く**

In [ ]:
import csv
import numpy as np
import os
import os.path
import sys
import glob
import pandas as pd
import codecs



disease_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/DiseaseInfo.csv'
patient_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/PatientInfo.csv'
connect_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/ZZZ_IDMap_20210709192453.csv'
previous_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/DiseaseInfo_all_connected.csv'
dst_path = '/content/drive/MyDrive/Deep_learning/Facephoto_LIST/DiseaseInfo_all_connected_new2.csv'

#最大表示列数の指定（ここでは50列を指定）
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)


#各ファイルを開く　※codec errorを回避
with codecs.open(disease_path, "r", "utf-8", "ignore") as file:
    df_data_disease = pd.read_csv(file, delimiter=",", header=None)

with codecs.open(patient_path, "r", "utf-8", "ignore") as file:
    df_data_patient = pd.read_csv(file, delimiter=",", header=None)
    Hushed_ID_list = df_data_patient.iloc[1:,0].tolist()
    for i in range(len(Hushed_ID_list)):
        Hushed_ID_list[i] = Hushed_ID_list[i].split("\t")[0]


with codecs.open(connect_path, "r", "utf-8", "ignore") as file:
    df_data_connect = pd.read_csv(file, delimiter=",", header=None)

with codecs.open(previous_path, "r", "utf-8", "ignore") as file:
    df_data_previous = pd.read_csv(file, delimiter=",", header=None)

#print(df_data_previous)
print("ppp")
print(Hushed_ID_list)

#**新しいCSVファイルを作成**

In [ ]:
#空のデータフレームを作成
#df_newList = pd.DataFrame(index=df_data_patient.index, columns=[]) #PatientInfoからindexのみを抜き出した空のデータフレーム
df_newList = pd.DataFrame(columns=['ID', 'Hushed_ID', 'img_num','age','gender', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10','11', '12', '13', '14', '15', '16', '17', '18', '19', '20'])

k=0
for i in Hushed_ID_list:
    print(i)
    df_data_connect[df_data_connect.iloc[0:,2] == i].iloc[0,0]
    id = df_data_connect[df_data_connect.iloc[0:,2] == i].iloc[0,0] #IDMapよりIDを抜き出す
    age = df_data_patient[df_data_patient.iloc[0:,0] == i].iloc[0,4] #PatientInfoより年齢を抜き出し（スペースよりうしろは削除）
    gender = df_data_patient[df_data_patient.iloc[0:,0] == i].iloc[0,2] #PatientInfoより性別を抜き出し（スペースよりうしろは削除）
    disease = list(set(df_data_disease[df_data_disease.iloc[0:,0] == i].iloc[:,3].tolist())) #DiseaseInfoより病名を抜き出す(リスト化して重複を削除）
    
    #病名リストの長さを20に揃える
    if len(disease) > 20:
        disease = disease[0:20]
    else:
        disease = disease + [""]*(20-len(disease))

    newRow = [int(id), str(i), "", age, gender] + disease  #リストを結合
    print(k)
    print(newRow)
    df_newList.loc[k] = newRow #結合したリストを行に追加
    k+=1


In [ ]:
#以前のリストに含まれるIDを除外したもの
df_newPatientList = df_newList[~df_newList.iloc[0:,0].isin(df_data_previous.iloc[0:,0])]
#通し番号を記載（以前のものが8210までだったので、それ以降）
df_newPatientList.iloc[:,2] = list(range(8211, 8211+len(df_newPatientList),1))

#以前のリストに含まれるIDが含まれるもの
df_oldPatientList = df_newList[df_newList.iloc[0:,0].isin(df_data_previous.iloc[0:,0])]
#以前の通し番号をIDを参照して転載（8210まで）
oldPatientID = list(df_oldPatientList.iloc[:,0])
for i in range(len(df_oldPatientList)):
    id = df_oldPatientList.iloc[i,0]
    number = df_data_previous[df_data_previous.iloc[:,0] == id].iloc[0,2]
    df_oldPatientList.iloc[i,2] = number

In [27]:
df_PatientList = pd.concat([df_oldPatientList, df_newPatientList])

#**CSVを保存**

In [28]:
df_PatientList.to_csv(dst_path, encoding='utf_8_sig', header=False, index = False)